In [82]:
r"""
T5Model预训练参数转换
"""

'\nT5Model预训练参数转换\n'

In [83]:
# select device
import os 
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [84]:
import torch, mindspore
import numpy as np
from transformers.models.t5 import modeling_t5 as pt
import mindnlp.models.t5 as m

In [85]:
path = r"/home/daiyuxin/ljm_script/codespace/t5_ckpt/t5-small"
config_path = f"{path}/t5-small_config.json"
pytorch_model_path = f"{path}/pytorch_model.bin"

In [86]:
# init config
import json
with open(config_path, encoding='utf-8') as config:
    config = json.load(config)

pt_config = pt.T5Config(**config)
pt_dict = torch.load(pytorch_model_path)
pt_model = pt.T5Model(pt_config)
pt_model.load_state_dict(pt_dict, False)

_IncompatibleKeys(missing_keys=['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'], unexpected_keys=['decoder.block.0.layer.1.EncDecAttention.relative_attention_bias.weight'])

In [87]:
# print pt_model parameters' name
pt_params = pt_model.state_dict()
for key in pt_params.keys():
    print(key)

shared.weight
encoder.embed_tokens.weight
encoder.block.0.layer.0.SelfAttention.q.weight
encoder.block.0.layer.0.SelfAttention.k.weight
encoder.block.0.layer.0.SelfAttention.v.weight
encoder.block.0.layer.0.SelfAttention.o.weight
encoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight
encoder.block.0.layer.0.layer_norm.weight
encoder.block.0.layer.1.DenseReluDense.wi.weight
encoder.block.0.layer.1.DenseReluDense.wo.weight
encoder.block.0.layer.1.layer_norm.weight
encoder.block.1.layer.0.SelfAttention.q.weight
encoder.block.1.layer.0.SelfAttention.k.weight
encoder.block.1.layer.0.SelfAttention.v.weight
encoder.block.1.layer.0.SelfAttention.o.weight
encoder.block.1.layer.0.layer_norm.weight
encoder.block.1.layer.1.DenseReluDense.wi.weight
encoder.block.1.layer.1.DenseReluDense.wo.weight
encoder.block.1.layer.1.layer_norm.weight
encoder.block.2.layer.0.SelfAttention.q.weight
encoder.block.2.layer.0.SelfAttention.k.weight
encoder.block.2.layer.0.SelfAttention.v.weight
encoder.

In [88]:
# init config
ms_config = m.T5Config(**config)
ms_model = m.T5Model(ms_config)

In [89]:
# print ms_model parameters' name
for key, param in ms_model.parameters_and_names():
    print(param.name)

decoder.embed_tokens.embedding_table
encoder.block.0.layer.0.SelfAttention.q.weight
encoder.block.0.layer.0.SelfAttention.k.weight
encoder.block.0.layer.0.SelfAttention.v.weight
encoder.block.0.layer.0.SelfAttention.o.weight
encoder.block.0.layer.0.SelfAttention.relative_attention_bias.embedding_table
encoder.block.0.layer.0.layer_norm.weight
encoder.block.0.layer.1.DenseReluDense.wi.weight
encoder.block.0.layer.1.DenseReluDense.wo.weight
encoder.block.0.layer.1.layer_norm.weight
encoder.block.1.layer.0.SelfAttention.q.weight
encoder.block.1.layer.0.SelfAttention.k.weight
encoder.block.1.layer.0.SelfAttention.v.weight
encoder.block.1.layer.0.SelfAttention.o.weight
encoder.block.1.layer.0.layer_norm.weight
encoder.block.1.layer.1.DenseReluDense.wi.weight
encoder.block.1.layer.1.DenseReluDense.wo.weight
encoder.block.1.layer.1.layer_norm.weight
encoder.block.2.layer.0.SelfAttention.q.weight
encoder.block.2.layer.0.SelfAttention.k.weight
encoder.block.2.layer.0.SelfAttention.v.weight
enco

In [90]:
import logging
def torch_to_mindspore(pth_file, size:str=None):
    try:
        import torch
    except:
        raise ImportError(f"'import torch' failed, please install torch by "
                          f"`pip install torch` or instructions from 'https://pytorch.org'")

    size = "mindspore" if not size else size # rename ckpt

    from mindspore import Tensor
    from mindspore.train.serialization import save_checkpoint

    logging.info('Starting checkpoint conversion.')
    ms_ckpt = []
    state_dict = torch.load(pth_file, map_location=torch.device('cpu'))

    for k, v in state_dict.items():
        if 'shared.weight' in k:
            k = k.replace('shared.weight', 'decoder.embed_tokens.embedding_table')
        if 'relative_attention_bias.weight' in k:
            k = k.replace('relative_attention_bias.weight', 'relative_attention_bias.embedding_table')
        ms_ckpt.append({'name': k, 'data': Tensor(v.numpy())})

    ms_ckpt_path = pth_file.replace('.bin','.ckpt')
    ms_ckpt_path = ms_ckpt_path.replace('pytorch',size)
    try:
        save_checkpoint(ms_ckpt, ms_ckpt_path)
    except:
        raise RuntimeError(f'Save checkpoint to {ms_ckpt_path} failed, please checkout the path.')

    return ms_ckpt_path

In [91]:
torch_to_mindspore("/home/daiyuxin/ljm_script/codespace/t5_ckpt/t5-small/pytorch_model.bin", "t5-small")

'/home/daiyuxin/ljm_script/codespace/t5_ckpt/t5-small/t5-small_model.ckpt'